ACTIAN VECTOR TUTORIAL
============

In this post, we're going to show you how to install and run big data analytics using [Actian Vector](https://www.actian.com/analytic-database/vector-smp-analytic-database/) columnar database using Jupyter Notebooks. Compared to traditional row-oriented databases, columnar databases like Vector really shine on analytics tasks where you need to do a lot of aggregate operations across many rows. Actian Vector in particular leverage modern processor architecture (multi-core and SIMD instructions) to heavily paralellize the workload and speed-up calculations.

Installing Actian Vector
------------------------

We'll work in a *Ubuntu Linux 64-bit 16.06 LTS* machine. We'll also assume you're using `bash`. Vector should work on any recent Linux distro so adapt the instructions below as needed. To install Actian Vector, we're going to follow the "Getting Started" section of the [official documentation](http://docs.actian.com/vector/5.0/index.html). First install the system dependencies, namely *LibAIO*

```bash
sudo apt install libaio1
```

First, download [Actian Vector Community Edition] (https://www.actian.com/lp/vector-community-edition/) that allows you 1TB of data without any time limit. After receiving the download link by email and fetching the install `tgz` file, uncompress and run the `install.sh` as super-user then follow instructions.

```bash
tar xvf ~/Downloads/actian-vector-5.0.0-405-community-linux-x86_64.tgz
cd actian-vector-5.0.0-405-community-linux-x86_64/
sudo ./install.sh
```

The default params will install under `/opt/Actian/VectorVW` (server ID `VW`) and create an `actian` user on your machine. Now let's run some post-installation commands to setup Vector as a system service.

```bash
sudo -u actian bash -c 'source ~actian/.ingVWsh && mkrc'
sudo cp /opt/Actian/VectorVW/ingres/files/rcfiles/actian-vectorVW /etc/init.d/
sudo systemctl enable actian-vectorVW
sudo systemctl start actian-vectorVW

```

Now you can use the regular `systemctl` commands to `start` and `stop` your `actian-vectoVW` service. Most administrative commands are run under the created `actian` user. We'll also `source` a small shell script (created during install) that setup `PATH` and other enviroment variables. Use the command below to run Vector commands as `actian` user.

```bash
sudo -u -i actian
source ~/.ingVWsh

```

To test your server is up and running, run the command below as `actian` user:

```bash
echo 'select 1\g' | sql iidbdb
```

This should output something like:

```
* Executing . . .


┌──────┐
│col1  │
├──────┤
│     1│
└──────┘
(1 row)
continue
* 
Your SQL statement(s) have been committed.
```
If you get an error like `E_LQ0001 Failed to connect to DBMS session.`, it means your server is not running or something went wrong during installation.


Creating a new test database 
----------------------------

We're going to use the `createdb` command to create a new `testdb` database and setup ODBC connection.

```bash
## as "actian" user
createdb testdb
iiodbcadmin add testdb
iiodbcadmin test testdb
```

The last command should output `iiodbcadmin: Connection was successful.` Now we're going to grant permission on the DB to your user using the SQL `CREATE USER` and `GRANT` statements. In the SQL commands below, replace `<user>` with the name of the OS user account that will run the Jupyter Notebook.

```bash
## as "actian" user
echo 'CREATE USER <user> WITH NOPROFILE, NOGROUP, NOEXPIRE_DATE, NOSECURITY_AUDIT \g' | sql iidbdb
echo 'GRANT ALL PRIVILEGES ON DATABASE testdb TO <user> \g' | sql iidbdb
```


Installing Python dependencies
------------------------------

If you don't already have a *Python 3.6* environment ready, install one using *Anaconda* package manager. Download Anaconda (or Miniconda if you prefer) from the [official download site](https://www.continuum.io/downloads). After download just run the command below and follow the instructions:

```bash
bash Anaconda3-4.4.0-Linux-x86_64.sh
```

The default (and assumed) install location is `~/anaconda3`. We're going to create a new virtual environment named `vector` just for this tutorial and activate it:

```bash
~/anaconda3/bin/conda create -y -n vector
source ~/anaconda3/bin/activate vector
```

This will give you a Bash prompt with `(vector)` string prepended. Now install the `conda` dependencies by running the command below:

```bash
## in (vector) virtualenv
conda install -y jupyter pandas pyodbc matplotlib seaborn tqdm
conda install -c conda-forge turbodbc=2.0.0
```

Running (this) Jupyter Notebook
-----------------------------------

Before running the Jupyter server, we need setup the environment variables as the user that will run the Jupyter notebook.

```bash
source ~actian/.ingVWsh
export ODBCSYSINI=/opt/Actian/VectorVW/ingres/files

## test we can connect to Vectian as the non-admin user
iiodbcadmin test testdb
```

Activate the `vector` virtual environment we created before, start the Jupyter server and open this `ActianVector.ipynb` notebook. 

```bash
## in the directory containing the .ipynb file and code
source ~/anaconda3/bin/activate vector

## in (vector) virtualenv
jupyter notebook .
```


Connecting to Vector using turbodbc
---------------------------------

The `turbodbc` is a DBAPI2 compatible Python library to connect to datasources providing an ODBC driver. The code below will connect to Vector and issue a sample SQL statement.

In [1]:
import turbodbc as odbc

# Connect to the pre-configured data source
conn = odbc.connect('testdb')

# Create a cursor and execute a statement.
cursor = conn.cursor()
cursor.execute('SELECT 1 as test_col')
res = cursor.fetchone()

print('Result:', res)

Result: [1]


Use Case: Smart energy metering logs
------------------------------------

Our test data will simulating a fleet of smart energy meters sending daily records consisting of energy consumption (in kWh), max and min voltage levels (in V) and a flag indicating an outage/brown-out. 

### Schema creation
The data is modeled after a typical "Star Schema" with a `metering_fact` fact table, and `customer_dim` and `date_dim` dimensions, as shown by the diagram below:

<p>
<img src="files/schema.png">
<p>

Let's create the schema now:

In [ ]:
date_dim_ddl = """
create table date_dim(
    skey integer4,
    date ansidate,
    day integer1,
    month integer1,
    year integer2,
    day_of_week integer1,
    week integer1,
    quarter integer1
)
"""

customer_dim_ddl = """
create table customer_dim(
    skey integer4,
    zipcode integer4,
    county varchar(255),
    state varchar(2)
)
"""

metering_fact_ddl = """
create table metering_fact(
    customer_skey integer4,
    date_skey integer4,
    consumption integer4,
    min_voltage integer2,
    max_voltage integer2,
    outage integer1
)
"""

cursor.execute(date_dim_ddl)
cursor.execute(customer_dim_ddl)
cursor.execute(metering_fact_ddl)
conn.commit()


We're not using any sort of index or constraint in the data model. First, Vector has it's own optimization for anaytical queries that generally doesn't require explict indexes. As for constraints, they have a performance impact on load and it's common to delegate data integrity to the ETL process instead of having it on the database on anaytical workloads.

### Data loading

The "customer" data is sampled randomly from a list of zip codes from the "New England" region of the US (Maine, Vermont, New Hampshire, Massachusetts, Rhode Island, and Connecticut) until we get about 500 thousand "customers". Then we generate daily measurements for each customer for two years (2015 and 2016) giving us about 356 million rows. The measurements are generated from a gaussian distribution with higher average consumption in winter and summer months.

The actual implementation code is in the `vector_tutorial.py` file.

In [2]:
from vector_tutorial import load_data
load_data(conn)

KeyboardInterrupt: 